In [25]:
import pandas as pd
import numpy as np
import math
import math
from pathlib import Path

from tournament import Tournament, Participant, Match

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
GAME_ID = 406  #4 Team 2021
GAME_ID = 359  # 6 Team 2016
# GAME_ID = 273  # 8 team 2012

In [29]:
league_settings_query = f"SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '{GAME_ID}'"
league_settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    league_settings_query
).drop_duplicates()

meta_query = "SELECT game_id, league_id, end_week FROM dev.league_metadata"
meta = DatabaseCursor(PATH).copy_data_from_postgres(meta_query).drop_duplicates()

league_settings = league_settings.merge(meta, how='inner', left_on=['game_id', 'league_id'], right_on=['game_id', 'league_id'])

num_playoff_teams = league_settings["num_playoff_teams"].values[0]
num_toliet_teams = league_settings["num_playoff_consolation_teams"].values[0]
playoff_start_week = league_settings["playoff_start_week"].values[0]
playoff_end_week = league_settings["end_week"].values[0]
playoff_weeks = range(playoff_start_week, playoff_end_week+1)

matchup_board_query = f"SELECT * FROM prod.playoff_board_{str(GAME_ID)}"
matchup_board = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(
    matchup_board_query
).drop_duplicates()

playoff_teams = list(
    matchup_board["team_name"][matchup_board["reg_season_rank"] <= num_playoff_teams].unique()
)
if num_toliet_teams != 0:
    toilet_teams = list(
        matchup_board["team_name"][matchup_board["reg_season_rank"] > num_playoff_teams].unique()
    )
else:
    toilet_teams = False

matchup_board.set_index(matchup_board['team_key'], drop=True, inplace=True)
matchup_board

Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '359'
Successfully pulled: SELECT game_id, league_id, end_week FROM dev.league_metadata
Successfully pulled: SELECT * FROM prod.playoff_board_359


,week,team_pts,team_pro_pts,ttl_pts_for_run,ttl_pro_pts_for_run,game_id,team_key,team_name,team_nickname,reg_season_rank
team_key,,,,,,,,,,


In [27]:
print(playoff_teams)
print(toilet_teams)

['Goons', 'Tebowners', 'Bro Lo El Cunado', 'ELE', 'Ruxin', 'The Fear Boners']
['ghost', 'Taco', 'Potatum', 'Cudde2', 'La Flama Blanca', 'Role players for day']


In [13]:
playoff_bracket = Tournament(playoff_teams)
if toilet_teams:
    toilet_bracket = Tournament(toilet_teams)

In [15]:
for match in playoff_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)

for match in playoff_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week+1)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week+1)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)
        
for match in playoff_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week+2)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week+2)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)

In [19]:
playoff_bracket.get_final()

In [22]:
for match in toilet_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)

for match in toilet_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week+1)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week+1)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)
        
for match in toilet_bracket.get_active_matches():
    right_comp = match.get_participants()[0].get_competitor()
    right_score = matchup_board['team_pts'][(matchup_board['team_name'] == right_comp) & (matchup_board['week'] == playoff_start_week+2)].values[0]
    left_comp = match.get_participants()[1].get_competitor()
    left_score = matchup_board['team_pts'][(matchup_board['team_name'] == left_comp) & (matchup_board['week'] == playoff_start_week+2)].values[0]
    if right_score > left_score:
        match.set_winner(right_comp)
    elif right_score < left_score:
        match.set_winner(left_comp)

In [ ]:
toilet_bracket.get_final()

In [5]:
# def tournament_round(df, week, teams):

#     tourney = Round(teams)
#     winner_list = []
#     loser_list = []
#     match_list = []
#     for match in tourney.get_active_matches():
#         home = match.get_participants()[0].get_competitor()
#         home_score = df[f'pts_{week}'][df['team_key'] == home].values[0]
#         away = match.get_participants()[1].get_competitor()
#         away_score = df[f'pts_{week}'][df['team_key'] == away].values[0]
#         match_list.append([home, away])

#         if home_score > away_score:
#             winner_list.append(home)
#             loser_list.append(away)
#         elif home_score < away_score:
#             winner_list.append(away)
#             loser_list.append(home)
#         else:
#             print('Error')
    
#     return winner_list, loser_list, match_list

# def results(contestants):
   
#     for i in range(len(contestants)):
#         lossing_team = matchups[i][1] if winners[i] == matchups[i][0] else matchups[i][0]
#         winning_team = matchups[i][1] if losers[i] == matchups[i][0] else matchups[i][0]

#         matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
#         matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
#         matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
#         matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#         matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#         matchup_board.at[winning_team, f'win_loss_{wk}'] = 'W'
#         matchup_board.at[winning_team, f'consol_{wk}'] = 'N'

#         matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
#         matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
#         matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
#         matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#         matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#         matchup_board.at[lossing_team, f'win_loss_{wk}'] = 'L'
#         matchup_board.at[lossing_team, f'consol_{wk}'] = 'Y'


In [6]:
# for wk in playoff_weeks:
#     if wk == playoff_start_week:
#         teams = list(matchup_board["team_key"][matchup_board["reg_season_rank"] <= num_playoff_teams])
#         winners, losers, matchups = tournament_round(matchup_board, wk, teams)

#         for i in range(len(winners)):
#             lossing_team = matchups[i][1] if winners[i] == matchups[i][0] else matchups[i][0]
#             winning_team = matchups[i][1] if losers[i] == matchups[i][0] else matchups[i][0]

#             matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
#             matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'win_loss_{wk}'] = f'W.{wk}.{i}'

#             matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
#             matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'L.{wk}.{i}'

#     elif wk == playoff_start_week+1:
#         matchup_board.sort_values([f'win_loss_{wk-1}'], inplace=True)
#         p_teams = list(matchup_board["team_key"][matchup_board[f"win_loss_{wk-1}"].str.contains('W') == True])
#         c_teams = list(matchup_board["team_key"][matchup_board[f"win_loss_{wk-1}"].str.contains('L') == True])
#         p_winners, p_losers, p_matchups = tournament_round(matchup_board, wk, p_teams)
#         c_winners, c_losers, c_matchups = tournament_round(matchup_board, wk, c_teams)
#         print(p_winners, p_losers, p_matchups, sep='\n')
#         print(c_winners, c_losers, c_matchups, sep='\n')

#         for i in range(len(p_winners)):
#             lossing_team = p_matchups[i][1] if p_winners[i] == p_matchups[i][0] else p_matchups[i][0]
#             winning_team = p_matchups[i][1] if p_losers[i] == p_matchups[i][0] else p_matchups[i][0]

#             matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
#             matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'win_loss_{wk}'] = f'W.{wk}.{i}'

#             matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
#             matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'L.{wk}.{i}'

#         for i in range(len(c_winners)):
#             lossing_team = c_matchups[i][1] if c_winners[i] == c_matchups[i][0] else c_matchups[i][0]
#             winning_team = c_matchups[i][1] if c_losers[i] == c_matchups[i][0] else c_matchups[i][0]

#             matchup_board.at[winning_team, f'opp_key_{wk}'] = lossing_team
#             matchup_board.at[winning_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == lossing_team].values[0]
#             matchup_board.at[winning_team, f'win_loss_{wk}'] = f'C.W.{wk}.{i}'

#             matchup_board.at[lossing_team, f'opp_key_{wk}'] = winning_team
#             matchup_board.at[lossing_team, f'opp_team_{wk}'] = matchup_board['team'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_mgr_{wk}'] = matchup_board['mgr'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pts_{wk}'] = matchup_board[f'pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'opp_pro_pts_{wk}'] = matchup_board[f'pro_pts_{wk}'][matchup_board['team_key'] == winning_team].values[0]
#             matchup_board.at[lossing_team, f'win_loss_{wk}'] = f'C.L.{wk}.{i}'



['273.l.777818.t.6', '273.l.777818.t.12']
['273.l.777818.t.7', '273.l.777818.t.2']
[['273.l.777818.t.7', '273.l.777818.t.6'], ['273.l.777818.t.12', '273.l.777818.t.2']]
['273.l.777818.t.1', '273.l.777818.t.10']
['273.l.777818.t.4', '273.l.777818.t.9']
[['273.l.777818.t.1', '273.l.777818.t.4'], ['273.l.777818.t.10', '273.l.777818.t.9']]


In [87]:
# # def test_tourney():

# __matches = []
# next_higher_power_of_two = int(
#     math.pow(2, math.ceil(math.log2(len(competitors_list))))
# )
# winners_number_of_byes = next_higher_power_of_two - len(competitors_list)
# incoming_participants = list(map(Participant, competitors_list))
# incoming_participants.extend([None] * winners_number_of_byes)
# num_of_rounds = int(math.ceil(math.log2(len(incoming_participants))))

# round_ = 1
# while round_ <= num_of_rounds:
#     if round_ == 1:
#         half_length = int(len(incoming_participants) / 2)
#         first = incoming_participants[0:half_length]
#         last = incoming_participants[half_length:]
#         last.reverse()
#         round_1_winners = []
#         round_1_losers = []
#         for participant_pair in zip(first, last):
#             if participant_pair[1] is None:
#                 round_1_winners.append(participant_pair[0])
#             elif participant_pair[0] is None:
#                 round_1_winners.append(participant_pair[1])
#             else:
#                 match = Match(participant_pair[0], participant_pair[1])
#                 round_1_winners.append(match.get_winner_participant())
#                 round_1_losers.append(match.get_loser_participant())
#                 __matches.append(match)
#         next_round = [round_1_winners, round_1_losers]
#         round_ += 1
    
#     elif round_ > 1:
#         while True:
#             if len(next_round[0]) > 1:
#                 bracket = next_round[0]
#                 half_length = int(len(bracket) / 2)
#                 first = bracket[0:half_length]
#                 last = bracket[half_length:]
#                 last.reverse()
#                 next_round_winners = []
#                 next_round_losers = []
#                 for participant_pair in zip(first, last):
#                     if participant_pair[1] is None:
#                         next_round_winners.append(participant_pair[0])
#                     elif participant_pair[0] is None:
#                         next_round_winners.append(participant_pair[1])
#                     else:
#                         match = Match(participant_pair[0], participant_pair[1])
#                         next_round_winners.append(match.get_winner_participant())
#                         next_round_losers.append(match.get_loser_participant())
#                         __matches.append(match)
#                 try:
#                     next_round.remove(bracket)
#                 except:
#                     continue
#                 next_round.append(next_round_winners)
#                 next_round.append(next_round_losers)
#                 next_round = [ele for ele in next_round if ele != []]
#                 if len(next_round) == len(competitors_list):
#                     break
#             round_ += 1

#     # return next_round, __matches, competitors_list